<a href="https://colab.research.google.com/github/luciekash/-Getting-Started-with-Text-Analysis/blob/main/Getting_Started_with_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="#4b76b7">To start practicing, you will need to make a copy of it. Go to File > Save a Copy in Drive. You can then use the new copy that will appear in the new tab.</font>


# AfterWork Data Science: Getting Started with NLP Project

**1. Defining the Question**

**a) Specifying the Data Analysis Question**

What is the question or problem you are trying to solve?

To create a model that can predict whether the sentiment of a tweet is positive or negative.



**b) Defining the Metric for Success**

How will you know your project has succeeded?

The desired accuracy of your model is 70%

**c) Understanding the context**

What is the background information surrounding the research question?

The management of a certain Marketing Firm would like to track the sentiments of their customers. This would help in shortening the amount of time that it takes to act on feedback.

**d) Recording the Experimental Design**

What steps will you take to answer the research question?


**● Text Cleaning/ Text Processing**

○ Removing all URLs/links

○ Replacing @ and # Characters

○ Feature Construction (No. of Punctuation Characters)

○ Removing Punctuation Characters

○ Feature Construction (Lowercase, Uppercase and Proper case words)

○ Conversion to Lowercase

○ Splitting Concatenated words

○ Spelling Correction

○ Feature Construction (Counting the no. of stop words/tweet)

○ Removing Stop words

○ Lemmatization

● Text Feature Engineering Techniques

○ Length of text

○ Word Count

○ Word density (Average no. of Words / Tweet)

○ Noun Count

○ Verb Count

○ Adjective Count

○ Adverb Count

○ Pronoun Count

○ Polarity

○ Subjectivity

○ Word Level N-Gram TF-IDF tweet_word_tfidf

○ Character Level N-Gram TF-IDF tweet_character_tfidf

***e) Data Relevance ***

Is the data provided relevant to the research question?


The provided dataset is appropriate for answering the research quesiton.

### Prerequisites

In [1]:
# Importing the required libraries
# ---
# 
import pandas as pd # library for data manipulation
import numpy as np  # librariy for scientific computations
import re           # regex library to perform text preprocessing
import string       # library to work with strings
import nltk         # library for natural language processing
import scipy        # scientific conputing 

### 1. Importing our Data

In [2]:
# Question: Given a new tweets, create a sentiment analysis model that will 
# predict whether a tweet will contain positive or negative sentiment.
# ---
# Dataset url = https://bit.ly/31kqByD 
# ---
#
df = pd.read_csv('https://bit.ly/31kqByD', encoding='latin-1')
df.head()

,Unnamed: 0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my d..."


### 2. Data Exploration

In [3]:
# We can determine the size of our dataset
# ---
#
df.shape

(10000, 7)

Seems this dataset will need some data cleaning i.e. columns. We also don't need some columns to perform create our model. We will drop those columns.

### 3. Data Preparation

#### Basic Data Cleaning Techniques

In [4]:
# We rename the columns for ease of referencing our columns later on
# ---
#
df.columns = ['id', 'target', 't_id', 'created_at', 'query', 'user', 'text']
df.head()

,id,target,t_id,created_at,query,user,text
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my d..."


In [5]:
# We retain the relevant columns by dropping the columns we don't need 
# for creating a sentiment analysis model. 
# ---
#
df = df.drop(['id', 't_id', 'created_at', 'query', 'user'], axis = 1)
df.head()

,target,text
0,0,Obama forges his Muslim alliance against the c...
1,4,Had the most spectacular prom ever but now my...
2,0,I am overwhelmed today taking a moment to eat...
3,0,@lindork Tres sad. I was totally a Max fan. #...
4,0,"Crap, I was counting down the hours until my d..."


In [6]:
# Understanding the distribution of target
# ---
#
df.target.value_counts() 

0    5067
4    4933
Name: target, dtype: int64

In [7]:
# Let's determine whether our columns have the right data types
# ---
#
df.dtypes

target     int64
text      object
dtype: object

In [8]:
# What values are in our target variable?
# ---
#
df.target.unique()

array([0, 4])

These are the two classes to which each document (text) belongs. The target value 0 means a text with a negative sentiment, while that of 4 means a text with a positive sentiment. 

In [9]:
# Let's check for missing values 
# ---
# 
df.isnull().sum()

target    0
text      0
dtype: int64

We don't have any missing values, so we are good to go.

#### Text Processing

In [10]:
# Text Cleaning: Removing all urls/links
# ---
# 
df['text'] =  df['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+','', str(x)))
df[['text']].head()

,text
0,Obama forges his Muslim alliance against the c...
1,Had the most spectacular prom ever but now my...
2,I am overwhelmed today taking a moment to eat...
3,@lindork Tres sad. I was totally a Max fan. #...
4,"Crap, I was counting down the hours until my d..."


In [11]:
# Text Cleaning: Removing @ and # characters or replace them with space
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.replace('#','')
df['text'] = df.text.str.replace('@','')
df[['text']].sample(5)

,text
3224,jayonpark yeah they are I never knew how expen...
5123,gimboland ! Half Jack is my favourite of all t...
6005,had a good day at school but is also hoping th...
4962,been asleep since like 8. no sleep for me toni...
8873,its weird people are tweeting that it the morn...


In [12]:
# Text Cleaning: Conversion to lowercase
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'].sample(5)


6779                       having beers then playing ball
8739     not happy about getting up this early for school
136             aamore i want to leave work early *sniff*
2327                              just ate too much again
2827    oral surgery &quot;part deux&quot; today. let ...
Name: text, dtype: object

In [13]:
# Text Cleaning: Splitting concatenated words
# ---
# Performing this step will take few minutes...
# ---
# YOUR CODE GOES BELOW
# 

# Installing wordnija and textblob
# ---
#
!pip3 install wordninja
!pip3 install textblob


# Importing those libraries
# ---
#
import wordninja 
from textblob import TextBlob

     |████████████████████████████████| 541 kB 5.1 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=1e0923e963d069c483c51893a417150cc487b8b0cfe783ac45b5df08f8040203
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


In [14]:
# Performing the split
# ---
#
df['text'] = df.text.apply(lambda x: wordninja.split(str(TextBlob(x))))  
df['text'] = df.text.str.join(' ')
df[['text']].sample(10) 

,text
6147,gonna s it in the sun and do homework i feel l...
6077,twitter won't let me change my background
4583,i dont like twitter its kinda wac k
4069,so i lied yesterday i did upgrade my iphone so...
5855,dan os aur 2000 hey you said you'd go to bed e...
1631,kirstie alley i really like to participate on ...
36,i hope i get all this homework done
1263,omg so bored no 1 is talking 2 day sigh
9346,broad com is trying to buy emu lex emu lex mak...
9037,damn twit game sucked yesterday maybe c uz of ...


In [16]:
# Text Cleaning: Removing punctuation characters
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.replace('[^\w\s]','')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [20]:
# Text Cleaning: Removing stop words
# ---
# YOUR CODE GOES BELOW
# 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['text'] = df.text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df[['text']].sample(5)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text
2364,ts elliott theres super follow sunday huh sweet
7462,messed colour tv yel yah williams looks orange...
8495,miss hazy awesome one best fan pic ive seen
7356,aw jair cant go see quo moon quo tomorrow
4239,miley cyrus shout please u never could text de...


In [24]:
# Text Cleaning: Lemmatization
# ---
# YOUR CODE GOES BELOW
#

# For lemmatization, we will need to download wordnet
#
nltk.download('wordnet')
from textblob import Word

# Lemmatizing our text
# ---
#
df['lemmatization'] = df.text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 
df[['text', 'lemmatization']].sample(10)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,lemmatization
8325,worry noone got one next question start 1 minu...,worry noone got one next question start 1 minu...
2123,hate forget password broadband router,hate forget password broadband router
5238,lakers fan please take bow like said 9 pm mamb...,lakers fan please take bow like said 9 pm mamb...
2962,finished copying pix dc blue j az experience,finished copying pix dc blue j az experience
5613,souljaboytellem th wu z p freestyle,souljaboytellem th wu z p freestyle
1915,weather girl 05 sign put thought would funny i...,weather girl 05 sign put thought would funny i...
3976,going start cooking dinner im hungry,going start cooking dinner im hungry
1135,wo love guess im sad somebody kno,wo love guess im sad somebody kno
28,ashley lt syf congrats bad couldnt watch perfo...,ashley lt syf congrats bad couldnt watch perfo...
8935,home day 1 son whose feeling great turtle ben ...,home day 1 son whose feeling great turtle ben ...


We won't remove numerics because we could loose meaning of our text if we lost the numerics. We could also further prepare our text by performing spelling correction but this is a resource intensive process that we will skip for now.

#### Feature Engineering Techniques 

In [29]:
# Custom Functions
# ---
#

# Avg. words
def avg_word(sentence):
  words = sentence.split()
  try:
    z = (sum(len(word) for word in words)/len(words))
  except ZeroDivisionError:
    z = 0 
  return z




# Subjectivity 
def get_subjectivity(tweet):
    try:
        textblob = TextBlob(unicode(tweet, 'utf-8'))
        subj = textblob.sentiment.subjectivity
    except:
        subj = 0.0
    return subj

# Polarity
def get_polarity(tweet):
    try:
        textblob = TextBlob(unicode(tweet, 'utf-8'))
        pol = textblob.sentiment.polarity
    except:
        pol = 0.0
    return pol

In [25]:
# Feature Construction: Length of tweet
# ---
# YOUR CODE GOES BELOW
#
df['length_of_tweet'] = df.text.str.len()


In [26]:
# Feature Construction: Word count 
# ---
# YOUR CODE GOES BELOW
# 
df['word_count'] = df.text.apply(lambda x: len(str(x).split(" ")))

In [30]:
# Feature Construction: Word density (Average no. of words / tweet)
# ---
# YOUR CODE GOES BELOW
#

df['avg_word_length'] = df.text.apply(lambda x: avg_word(x))

In [35]:
# Feature Construction: Noun count
# ---
# YOUR CODE GOES BELOW
#
# First, we will download the punkt and the averaged_perceptron_tagger into our notebook environment. 
# which will allow us to find the part of speech tags.
# ---
#
# Library for Noun count
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# We create the function to check and get the part of speech tag count of a words in a given sentence
# Noun count
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [33]:
# Noun Count
# ---
# YOUR CODE GOES BELOW
#
df['noun_count'] = df.text.apply(lambda x: pos_check(x, 'noun'))

In [34]:
# Feature Construction: Verb count
# ---
# YOUR CODE GOES BELOW
#
df['verb_count'] = df.text.apply(lambda x: pos_check(x, 'verb'))

In [36]:
# Feature Construction: Adjective count / Tweet
# ---
# YOUR CODE GOES BELOW
#
df['adj_count'] = df.text.apply(lambda x: pos_check(x, 'adj'))

In [37]:
# Feature Construction: Adverb count / Tweet
# ---
# YOUR CODE GOES BELOW
#
df['adv_count'] = df.text.apply(lambda x: pos_check(x, 'adv'))

In [39]:
# Feature Construction: Pronoun 
# ---
# YOUR CODE GOES BELOW
#
df['pron_count'] = df.text.apply(lambda x: pos_check(x, 'pron'))

In [41]:
# Feature Construction: Subjectivity
# ---
# YOUR CODE GOES BELOW
# 
df['subjectivity'] = df.text.apply(get_subjectivity)
 

In [42]:
# Feature Construction: Polarity
# ---
# YOUR CODE GOES BELOW
# 
df['polarity'] = df.text.apply(get_polarity)

In [44]:
# Feature Construction: Word Level N-Gram TF-IDF Feature 
# ---
from sklearn.feature_extraction.text import TfidfVectorizer
# YOUR CODE GOES BELOW
#
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', ngram_range=(1,3),  stop_words= 'english')
df_word_vect = tfidf.fit_transform(df.text)

In [45]:
# Feature Construction: Character Level N-Gram TF-IDF Feature
# ---
# YOUR CODE GOES BELOW
# 
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='char', ngram_range=(1,3),  stop_words= 'english')
df_char_vect = tfidf.fit_transform(df.text)

In [48]:
# Let's prepare the constructed features for modeling
# ---
#
X_metadata = np.array(df.iloc[:, 3:12])
X_metadata

array([[67.        , 11.        ,  5.18181818, ...,  2.        ,
         0.        ,  0.        ],
       [81.        , 12.        ,  5.83333333, ...,  1.        ,
         0.        ,  0.        ],
       [40.        ,  6.        ,  5.83333333, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [45.        ,  8.        ,  4.75      , ...,  2.        ,
         0.        ,  0.        ],
       [34.        ,  6.        ,  4.83333333, ...,  1.        ,
         0.        ,  0.        ],
       [44.        , 10.        ,  3.5       , ...,  0.        ,
         0.        ,  0.        ]])

In [49]:
# We combine our two tfidf (sparse) matrices and X_metadata
# ---
#
X = scipy.sparse.hstack([df_word_vect, df_char_vect,  X_metadata])
X

<10000x2009 sparse matrix of type '<class 'numpy.float64'>'
	with 938044 stored elements in COOrdinate format>

In [50]:
# Getting our response variable
# ---
#
y = np.array(df.iloc[:, 0])
y

array([0, 4, 0, ..., 0, 4, 0])

### 4. Data Modelling

During this step, we will use machine learning algorithms to train and test our sentiment analysis models.

In [51]:
# Splitting our data
# ---
#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
# Fitting our model
# ---
#

# Importing the algorithms
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

nb_classifier = MultinomialNB() 
lr_classifier = LogisticRegression(max_iter=1000) 

# Training our model
nb_classifier.fit(X_train, y_train) 
lr_classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [53]:
# Making predictions
# ---
#
y_predict_nb = nb_classifier.predict(X_test) 
y_predict_lr = lr_classifier.predict(X_test)

In [54]:
# Evaluating the Models
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Accuracy scores
# ---
#
print("Naive Bayes Classifier:\n", accuracy_score(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", accuracy_score(y_test, y_predict_lr))

Naive Bayes Classifier:
 0.7265
Logistic Regression Classifier: 
 0.7315


In [55]:
# Confusion matrices
# ---
# 
print("Naive Bayes Classifier: \n", confusion_matrix(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", confusion_matrix(y_test, y_predict_lr))

Naive Bayes Classifier: 
 [[760 290]
 [257 693]]
Logistic Regression Classifier: 
 [[759 291]
 [246 704]]


In [56]:
# Classification Reports
# ---
#
print("Naive Bayes Classifier: \n", classification_report(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", classification_report(y_test, y_predict_lr))

Naive Bayes Classifier: 
               precision    recall  f1-score   support

           0       0.75      0.72      0.74      1050
           4       0.70      0.73      0.72       950

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

Logistic Regression Classifier: 
               precision    recall  f1-score   support

           0       0.76      0.72      0.74      1050
           4       0.71      0.74      0.72       950

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000



**Evaluation our Models**

* **Accuracy:** the percentage of texts that were assigned the correct topic.
* **Precision:** the percentage of texts the classifier classified correctly out of the total number of texts it predicted for each topic
* **Recall:** the percentage of texts the model predicted for each topic out of the total number of texts it should have predicted for that topic.
* **F1 Score:** the average of both precision and recall.

To improve our model, we can try perfoming other text processing techniques that would better prepare our data for fitting our model. We can also use different vectorizing techniques, implement other machine learning models and perform hyperparameter tuning.

### 5. Recommendations


Our best model had an accuracy of 73.25% and use it for classifying newer tweets. We can improve this performance by performing hyperparameter tuning and feature engineering methods. 